In [1]:
import numpy as np
from datasets import load_dataset
from keras_preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import (
    Input,
    Embedding,
    Conv1D,
    MaxPooling1D,
    Flatten,
    Reshape,
    LSTM,
    Dense
)
from keras.models import Model
from keras.optimizers import Adam

d:\py12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = load_dataset("k1tub/sentiment_dataset")  

print("Dataset splits:", data)

Dataset splits: DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'src'],
        num_rows: 290458
    })
})


In [3]:
train_texts = data["train"]["text"]
train_labels = np.array(data["train"]["label"])

train_labels = np.clip(train_labels, 0, 2)


In [4]:
max_words = 30000
max_len = 100

In [5]:
tokenizer = Tokenizer(num_words=max_words, oov_token="<UNK>")
tokenizer.fit_on_texts(train_texts)


In [6]:
X_train = pad_sequences(
    tokenizer.texts_to_sequences(train_texts),
    maxlen=max_len
)
y_train = train_labels

In [7]:
inputs = Input(shape=(max_len,))

x = Embedding(max_words, 100, input_length=max_len)(inputs)
x = Conv1D(128, 3, activation="relu")(x)
x = MaxPooling1D(2)(x)
x = Flatten()(x)

x = Reshape((49, 128))(x)

x = LSTM(64)(x)

x = Dense(64, activation="relu")(x)
outputs = Dense(3, activation="softmax")(x)

model = Model(inputs, outputs)
model.compile(
    optimizer=Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

d:\py12\Lib\site-packages\keras\src\layers\core\embedding.py:100: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 100)       │     3,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 98, 128)        │        38,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 49, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 49, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,092,291 (11.80 MB)

 Trainable params: 3,092,291 (11.80 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:

model.fit(
    X_train,
    y_train,
    epochs=5,
    batch_size=32,
    validation_split=0.1
)

Epoch 1/5
8170/8170 ━━━━━━━━━━━━━━━━━━━━ 270s 32ms/step - accuracy: 0.6955 - loss: 0.6637 - val_accuracy: 0.4815 - val_loss: 1.2480
Epoch 2/5
8170/8170 ━━━━━━━━━━━━━━━━━━━━ 275s 34ms/step - accuracy: 0.7585 - loss: 0.5461 - val_accuracy: 0.4872 - val_loss: 1.1780
Epoch 3/5
8170/8170 ━━━━━━━━━━━━━━━━━━━━ 278s 34ms/step - accuracy: 0.8095 - loss: 0.4477 - val_accuracy: 0.4144 - val_loss: 1.7793
Epoch 4/5
8170/8170 ━━━━━━━━━━━━━━━━━━━━ 284s 35ms/step - accuracy: 0.8624 - loss: 0.3368 - val_accuracy: 0.4641 - val_loss: 1.8175
Epoch 5/5
8170/8170 ━━━━━━━━━━━━━━━━━━━━ 270s 33ms/step - accuracy: 0.9024 - loss: 0.2467 - val_accuracy: 0.4351 - val_loss: 2.3161


In [10]:


labels_map = {0: "neutral", 1: "positive", 2: "negative"}

texts_to_predict = [
    "Это просто ужасно, сервис отвратительный",
    "Ну нормально, пойдет",
    "Мне очень понравилось, всё супер"
]

seq = tokenizer.texts_to_sequences(texts_to_predict)
pad = pad_sequences(seq, maxlen=max_len)

preds = model.predict(pad)

for text, pred in zip(texts_to_predict, preds):
    idx = np.argmax(pred)
    print(f"Text: {text}")
    print(f"→ {labels_map[idx]} ({pred[idx]:.2%})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Text: Это просто ужасно, сервис отвратительный
→ negative (99.01%)
Text: Ну нормально, пойдет
→ neutral (52.73%)
Text: Мне очень понравилось, всё супер
→ positive (98.82%)
